In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import json
from datetime import datetime

import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')


# this enables relative path imports
import os
from dotenv import load_dotenv
load_dotenv()
_PROJECT_PATH: str = os.environ["_project_path"]
_PICKLED_DATA_FILENAME: str = os.environ["_pickled_data_filename"]

import sys
from pathlib import Path
project_path = Path(_PROJECT_PATH)
sys.path.append(str(project_path))

import config_v2 as cfg

from library_report_v2 import Cleaning as cln
from library_report_v2 import Graphing as grp
from library_report_v2 import Processing as pro
from library_report_v2 import Configuration as repcfg

Baseline en config_v2.py: ['2024-04-01', '2024-08-26']
Study en config_v2.py: ['2024-08-26', '2024-09-02']


In [2]:
def show_response_contents(df):
    print("The response contains:")
    print(json.dumps(list(df['variable'].unique()), sort_keys=True, indent=4))
    print(json.dumps(list(df['device'].unique()), sort_keys=True, indent=4))

In [3]:
df = pd.read_pickle(project_path / 'data' / _PICKLED_DATA_FILENAME)
#show_response_contents(df)

In [4]:
df = df.sort_values(by=['variable','datetime'])
df = pro.datetime_attributes(df)

#df_bl, df_st = pro.split_into_baseline_and_study(df, baseline=cfg.BASELINE, study=cfg.STUDY, inclusive='left')
#df_bl['Periodo'] = 'Baseline'
#df_st['Periodo'] = 'Estudio'

#past_w = df_bl.loc[cfg.PAST_WEEK[0]:cfg.PAST_WEEK[1]]

In [13]:
#Del dataframe completo (baselin-study) se saca solo la variable energía activa
front = df.query("variable == 'front-consumo-activa'").copy()

#Se agrupan los datos por intervalos de una hora
front_hour = front.groupby(by=["variable", "device_name"]).resample('1h').sum().round(2).reset_index().set_index('datetime')
front_hour = pro.datetime_attributes(front_hour)


In [16]:
front_hour = front_hour[front_hour['year'] == 2024]


In [15]:
# Seleccionar columnas específicas directamente
front_hour = front_hour[['variable', 'device_name', 'value', 'hour', 'day', 'month', 'year', 'dow']]


In [18]:
front_hour.to_clipboard(index=True, header=True)

In [ ]:
#front_hour = front.groupby(['device_name', 'hour']).agg({'value': 'mean', 'day': 'first', 'dow':'first',
#       'cont_dow':'first', 'week':'first', 'year':'first' }).reset_index().round(2)